In [ ]:
from peft import PeftModel

base_model_name = "./model/Qwen2.5-7B"
lora_path = "./output/qwen2p5_7b/checkpoint-702"

# 1. 加载基础模型
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name).to(device)

# 2. 加载 LoRA
lora_model = PeftModel.from_pretrained(base_model, lora_path).to(device)

# 3. 合并 LoRA 到 base_model
lora_model.merge_and_unload()  # LoRA 权重直接写入 base_model


In [ ]:
prompt = "指令: 你爸爸是谁？\n回答:"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    output = base_model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.8,
        top_p=0.9
    )

text = tokenizer.decode(output[0], skip_special_tokens=True)
print(text)

In [ ]:
merged_model_dir = "./qwen2.5_7B_lora_merged"
base_model.save_pretrained(merged_model_dir)
tokenizer.save_pretrained(merged_model_dir)

In [ ]:
# ! ms-b4990507-632e-49f8-9ec8-70461fa8b180

In [ ]:
from modelscope.hub.api import HubApi

YOUR_ACCESS_TOKEN = 'ms-b4990507-632e-49f8-9ec8-70461fa8b180'
api = HubApi()
api.login(YOUR_ACCESS_TOKEN)

In [ ]:
from modelscope.hub.constants import Licenses, ModelVisibility

owner_name = 'nev8rz'
model_name = 'huanhuanchat_base_qwen2.5_7B'
model_id = f"{owner_name}/{model_name}"

api.create_model(
    model_id,
    visibility=ModelVisibility.PUBLIC,
    license=Licenses.APACHE_V2,
    chinese_name="嬛嬛"
)

In [ ]:
api.upload_folder(
    repo_id=f"{owner_name}/{model_name}",
    folder_path='./qwen2.5_7B_lora_merged/',
    commit_message='聪明一点的嬛嬛',
)